## Import the libraries you need.
Other internal functions(nilean, keras, etc..) dosen't need to be imported.
Because mb-mvpa is wrapping.

In [1]:
from mbmvpa.preprocessing.bids import bids_preprocess
from mbmvpa.preprocessing.events import events_preprocess
from mbmvpa.data.loader import prepare_dataset
from mbmvpa.utils.example_utils import load_example_data

In [2]:
root = load_example_data('tom')

data load success! (tom)


In [3]:
X, voxel_mask, layout = bids_preprocess(root, smoothing_fwhm=None, zoom=(2, 2, 2), ncore=2, nthread=4)

bids preprocessing done!                          :  83%|████████▎ | 5/6 [00:18<00:03,  3.18s/it]INFO:root:time elapsed: 0.32 minutes
INFO:root:result
masking data shape: (46, 55, 46)

bids preprocessing done!                          : 100%|██████████| 6/6 [00:18<00:00,  3.17s/it]


In [4]:
def example_tom_adjust_columns(row):
    ## rename data in a row to the name which can match hbayesdm.ra_prospect requirements ##
    row["gamble"] = 1 if row["respcat"] == 1 else 0
    row["cert"] = 0
    return row

In [5]:
def example_tom_condition(row):
    # include all trial data
    return True

In [6]:
def example_tom_modulation(row, param_dict):
    ## calculate subjectives utility for choosing Gamble over Safe option
    ## prospect theory with loss aversion and risk aversion is adopted
    modulation = (row["gain"] ** param_dict["rho"]) - (param_dict["lambda"] * (row["loss"] ** param_dict["rho"]))
    row["modulation"] = modulation
    return row

In [7]:
dm_model, df_events, signals, time_masks, _ = \
    events_preprocess(root,
                      preprocess=example_tom_adjust_columns,
                      condition=example_tom_condition,
                      modulation=example_tom_modulation,
                      dm_model='ra_prospect')

hbayesdm doing (model: ra_prospect)..                      :  50%|█████     | 3/6 [00:03<00:04,  1.65s/it]INFO:numexpr.utils:Note: detected 88 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 88 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


Using cached StanModel: cached-ra_prospect-pystan_2.19.1.1.pkl

Model  = ra_prospect
Data   = <pandas.DataFrame object>

Details:
 # of chains                    = 4
 # of cores used                = 1
 # of MCMC samples (per chain)  = 4000
 # of burn-in samples           = 1000
 # of subjects                  = 2
 # of (max) trials per subject  = 256

Using cached StanModel: cached-ra_prospect-pystan_2.19.1.1.pkl


calculating modulation..                          :  67%|██████▋   | 4/6 [12:01<03:37, 108.98s/it]         /home/mybirth0407/anaconda3/envs/mvpa/lib/python3.7/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)


Index(['onset', 'duration', 'parametric loss', 'distance from indifference',
       'parametric gain', 'gain', 'loss', 'ptval', 'respnum', 'respcat',
       'responsetime', 'subjid', 'run', 'gamble', 'cert'],
      dtype='object')
['onset', 'duration', 'parametric loss', 'distance from indifference', 'parametric gain', 'gain', 'loss', 'PTval', 'respnum', 'respcat', 'response_time', 'subjID', 'run', 'gamble', 'cert']
************************************
**** Model fitting is complete! ****
************************************


KeyError: 'rho'

## model

In [ ]:
from model_based_mvpa.models.regressor import *
from model_based_mvpa.utils.coef2map import *
import time

In [ ]:
X, y, voxel_mask = prepare_dataset(root=layout.derivatives["fMRIPrep"].root)

In [ ]:
coefs = mlp_regression_v2(X, y,
                       layer_dims=[1024, 1024],
                       activation_func='linear',
                       dropout_rate=0.5,
                       epochs=100,
                       patience=10,
                       batch_size=64,
                       N=3,
                       verbose=1)

task_name = 'tom2007_mlp_v2'

result = get_map(coefs, voxel_mask, task_name, map_type='z', save=True, save_path='./results' sigma=1)
print(time.strftime('%c', time.localtime(time.time())))

In [ ]:
coefs = penalized_linear_regression(X, y,
                             alpha=0.001,
                             lambda_par=2.0,
                             epochs=100,
                             patience=30,
                             batch_size=256,
                             N=30,
                             verbose=1)

task_name = 'tom2007_penalized_linear'

result = get_map(coefs, voxel_mask, task_name, map_type='z', save=True, save_path='./results' sigma=1)
print(time.strftime('%c', time.localtime(time.time())))

In [ ]:
coefs = elasticnet(X, y,max_use_sample_N=10000,
             alpha=0.001,
             n_jobs=16,
             N=3,
             verbose=3)

task_name = 'tom2007_elasticnet'

result = get_map(coefs, voxel_mask, task_name, map_type='z', save=True, save_path='./results' sigma=1)
print(time.strftime('%c', time.localtime(time.time())))